In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

from bokeh.tile_providers import STAMEN_TERRAIN

BokehJS successfully loaded.

In [2]:
import datashader as ds
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

import orca
import datasources
import variables
import models
orca.run(['build_networks'])

Model template is: zone. Operating on zones with zone_id.
Running step 'build_networks'
Number of nodes is 27786.
Number of edges is 37525.
Precomputing network for distance 40000.
Network precompute starting.
Network precompute done.
Time to execute step 'build_networks': 6.53 s
Total time to execute iteration 1 with iteration value None: 6.53 s


In [3]:
df = orca.get_table('households').to_frame()

In [4]:
region = x_range, y_range = ((df.x.min(),df.x.max()), (df.y.min(),df.y.max()))

plot_width  = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p
    
options = dict(line_color=None, fill_color='blue', size=5)

In [5]:




background = "black"
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="black"))

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'x', 'y',  ds.count('persons'))
    img = tf.shade(agg, cmap=Hot, how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

p = base_plot(background_fill_color=background)
#p.add_tile(STAMEN_TERRAIN)
export(create_image(*region),"regionT_hot")
InteractiveImage(p, create_image)

